## Traer datos

In [4]:
import os
import stat
import paramiko
import pandas as pd
from pathlib import Path

# ──────────────────────────────────────────────────────────────────────────────
# 1) PARÁMETROS 
# ──────────────────────────────────────────────────────────────────────────────
HOST, PORT = "172.16.70.36", 22
USER, PASS = "haider_bello", "0MJyrGDy96R0LQ"     # En prod, usa variables de entorno
ROOT_REM = "/u01/bk/ftp/telefonia/ContactVox_CLTIENE/tasks"

COL = "nombre_audio"

DEST = Path(r"C:\Descargas\Contactvox_ultimo33333_ventas_1_07_2026")
DEST.mkdir(parents=True, exist_ok=True)

XLIST = r"C:\Users\haider_bello\Downloads\Información llamadas de venta y de servicio del 12 de Diciembre al 2 de Enero (1).xlsx"

# ── CONFIG DE HOJAS (elige UNA opción) ────────────────────────────────────────
# Opción A: una hoja por nombre
SHEET_NAME = "Ventas"   # <-- cambia por el nombre exacto
SHEET_NAMES = None                  # no tocar

# Opción B: varias hojas por nombre (descomenta estas 2 líneas y comenta la Opción A)
# SHEET_NAME = None
# SHEET_NAMES = ["VENTAS (SALIENTES)", "SERVICIO (ENTRANTES)"]

# Opción C: todas las hojas (descomenta estas 2 líneas y comenta la Opción A)
# SHEET_NAME = None
# SHEET_NAMES = "ALL"

# Extensiones permitidas
EXTS = {".wav"}

# Progreso (cada cuántos archivos revisados reportar)
PROGRESO_CADA = 100

# Directorios a ignorar explícitamente (opcionales, acelera)
IGNORAR_DIRS = {".cache", ".config", ".ssh"}

# Límite defensivo de profundidad
MAX_DEPTH = 50


# ──────────────────────────────────────────────────────────────────────────────
# 2) UTILIDADES
# ──────────────────────────────────────────────────────────────────────────────
def ruta_unica(p: Path) -> Path:
    """Evita sobrescrituras: archivo, archivo_1, archivo_2, ..."""
    if not p.exists():
        return p
    i = 1
    while (p.with_stem(f"{p.stem}_{i}")).exists():
        i += 1
    return p.with_stem(f"{p.stem}_{i}")

def base_sin_ext(nombre: str) -> str:
    """Nombre base sin extensión, en minúsculas."""
    root, _ext = os.path.splitext(nombre)
    return root.strip().lower()

def es_extension_valida(nombre: str) -> bool:
    """True si la extensión está en EXTS (case-insensitive)."""
    _, ext = os.path.splitext(nombre)
    return ext.lower() in EXTS

def normaliza_objetivo(s: str) -> str:
    """
    Deja solo el primer token (por si hay comentarios), quita extensión si viene,
    y pasa a minúsculas.
    """
    tok = str(s).strip().split()[0]
    if tok.lower().endswith(tuple(EXTS)):
        root, _ = os.path.splitext(tok)
        tok = root
    return tok.lower()

def safe_listdir_attr(sftp: paramiko.SFTPClient, remote_path: str):
    """listdir_attr con manejo de errores/permiso."""
    try:
        return sftp.listdir_attr(remote_path)
    except Exception as e:
        print(f"⚠️  No se pudo listar: {remote_path} ({e})")
        return []

def cargar_columna_excel(xfile: str, col: str, sheet_name=None, sheet_names=None) -> pd.Series:
    """
    Lee la columna 'col' desde:
      - una hoja (sheet_name="..."),
      - varias hojas (sheet_names=[...]),
      - o todas (sheet_names="ALL"),
    y concatena todo en una sola Series.
    """
    engine = "openpyxl"

    if sheet_names == "ALL":
        xls = pd.ExcelFile(xfile, engine=engine)
        hojas = xls.sheet_names
    elif isinstance(sheet_names, list):
        hojas = sheet_names
    elif sheet_name is not None:
        hojas = [sheet_name]
    else:
        # default: primera hoja
        xls = pd.ExcelFile(xfile, engine=engine)
        hojas = [xls.sheet_names[0]]

    series_list = []
    for sh in hojas:
        df_sh = pd.read_excel(xfile, sheet_name=sh, engine=engine)
        if col not in df_sh.columns:
            print(f"⚠️  La hoja '{sh}' NO tiene la columna '{col}'. Se omite esa hoja.")
            continue
        series_list.append(df_sh[col].astype(str))

    if not series_list:
        raise ValueError(f"No se encontró la columna '{col}' en ninguna hoja seleccionada.")

    return pd.concat(series_list, ignore_index=True)


# ──────────────────────────────────────────────────────────────────────────────
# 3) LEE Y NORMALIZA LA LISTA DE OBJETIVOS (DESDE HOJA(S))
# ──────────────────────────────────────────────────────────────────────────────
df = cargar_columna_excel(
    XLIST,
    col=COL,
    sheet_name=SHEET_NAME,
    sheet_names=SHEET_NAMES
)

df = df.dropna().astype(str)

OBJETIVOS = set(df.map(normaliza_objetivo))   # bases sin extensión
NO_ENCONTRADOS = set(OBJETIVOS)               # se vaciará cuando se descarguen
print(f"👉 {len(OBJETIVOS):,} nombres objetivo cargados (normalizados).")


# ──────────────────────────────────────────────────────────────────────────────
# 4) RECORRIDO + DESCARGA (con prevención de ciclos/symlinks)
# ──────────────────────────────────────────────────────────────────────────────
VISITADOS = 0
VISITED_DIRS = set()

def buscar_descargar(sftp: paramiko.SFTPClient, ruta: str, depth: int = 0):
    """Recorre recursivamente 'ruta' y descarga coincidencias en OBJETIVOS."""
    global VISITADOS

    # Normaliza para detectar ciclos
    try:
        ruta_norm = sftp.normalize(ruta)
    except Exception:
        ruta_norm = ruta

    if ruta_norm in VISITED_DIRS:
        return
    VISITED_DIRS.add(ruta_norm)

    if depth > MAX_DEPTH:
        print(f"⛔ Profundidad máxima alcanzada en: {ruta_norm}")
        return

    for entry in safe_listdir_attr(sftp, ruta):
        nombre = entry.filename
        if nombre in IGNORAR_DIRS:
            continue

        remoto = f"{ruta.rstrip('/')}/{nombre}"
        mode = entry.st_mode

        # No seguir symlinks
        if stat.S_ISLNK(mode):
            continue

        if stat.S_ISDIR(mode):
            buscar_descargar(sftp, remoto, depth + 1)
            continue

        # Archivo regular
        VISITADOS += 1
        if PROGRESO_CADA and VISITADOS % PROGRESO_CADA == 0:
            print(f"⏳ Revisados {VISITADOS:,} archivos...")

        if not es_extension_valida(nombre):
            continue

        nombre_base = base_sin_ext(nombre)
        if nombre_base in OBJETIVOS:
            local = ruta_unica(DEST / nombre)
            try:
                sftp.get(remoto, str(local))
                NO_ENCONTRADOS.discard(nombre_base)
                print(f"⬇️  {remoto}  →  {local.name}")
            except Exception as e:
                print(f"❌ Error al descargar {remoto}: {e}")


# ──────────────────────────────────────────────────────────────────────────────
# 5) CONEXIÓN SFTP Y EJECUCIÓN
# ──────────────────────────────────────────────────────────────────────────────
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    if not PASS:
        raise ValueError("Falta la contraseña. Define la variable de entorno SFTP_PASS o asigna PASS en el script.")

    client.connect(HOST, port=PORT, username=USER, password=PASS, timeout=30)

    with client.open_sftp() as sftp:
        print("✅ Conectado al servidor.")
        buscar_descargar(sftp, ROOT_REM, depth=0)
finally:
    client.close()


# ──────────────────────────────────────────────────────────────────────────────
# 6) REPORTE FINAL
# ──────────────────────────────────────────────────────────────────────────────
descargados = len(OBJETIVOS) - len(NO_ENCONTRADOS)
print(f"\n🎯 Archivos descargados: {descargados:,}")
print(f"❌ No encontrados   : {len(NO_ENCONTRADOS):,}")

if NO_ENCONTRADOS:
    out_xlsx = DEST / "no_encontrados.xlsx"
    pd.DataFrame(sorted(NO_ENCONTRADOS), columns=["no_encontrado"]).to_excel(out_xlsx, index=False)
    print(f"📁 Se guardó la lista de 'no encontrados' en: {out_xlsx}")

print("✅ Proceso terminado. Archivos guardados en:", DEST)

👉 3,983 nombres objetivo cargados (normalizados).
✅ Conectado al servidor.
⏳ Revisados 100 archivos...
⏳ Revisados 200 archivos...
⏳ Revisados 300 archivos...
⏳ Revisados 400 archivos...
⏳ Revisados 500 archivos...
⏳ Revisados 600 archivos...
⏳ Revisados 700 archivos...
⏳ Revisados 800 archivos...
⏳ Revisados 900 archivos...
⏳ Revisados 1,000 archivos...
⏳ Revisados 1,100 archivos...
⏳ Revisados 1,200 archivos...
⏳ Revisados 1,300 archivos...
⏳ Revisados 1,400 archivos...
⏳ Revisados 1,500 archivos...
⏳ Revisados 1,600 archivos...
⏳ Revisados 1,700 archivos...
⏳ Revisados 1,800 archivos...
⏳ Revisados 1,900 archivos...
⏳ Revisados 2,000 archivos...
⏳ Revisados 2,100 archivos...
⏳ Revisados 2,200 archivos...
⏳ Revisados 2,300 archivos...
⏳ Revisados 2,400 archivos...
⏳ Revisados 2,500 archivos...
⏳ Revisados 2,600 archivos...
⏳ Revisados 2,700 archivos...
⏳ Revisados 2,800 archivos...
⏳ Revisados 2,900 archivos...
⏳ Revisados 3,000 archivos...
⏳ Revisados 3,100 archivos...
⏳ Revisados 3

## pasar a txt 

In [ ]:

# !pip install faster-whisper torch --upgrade
import os
import time
import math
import threading
from queue import Queue
from pathlib import Path
from faster_whisper import WhisperModel
import torch

# ───── RUTAS (AJUSTA) ───────────────────────────────────────────────────────────
INPUT_ROOT = Path(r"C:\descargas\Contactvox_ultimo33333_ventas_1_07_2026")
OUTPUT_DIR = Path(r"C:\descargas\Contactvox_ultimo33333_ventas_1_07_2026\txt")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

AUDIO_EXTS = {".wav", ".mp3", ".m4a", ".flac", ".ogg", ".aac", ".wma"}

# ───── DETECCIÓN GPU / CONFIG ──────────────────────────────────────────────────
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "Para 90% de uso necesitas GPU. No se detectó CUDA."
torch.set_float32_matmul_precision("medium")

MODEL_NAME   = "medium"
COMPUTE_TYPE = "float16"           # FP16 en GPU = rápido
BEAM_SIZE    = 1                   # 1 = más veloz
LANG         = "es"

# Estimar cuántas instancias caben en 12 GB (aprox medium FP16 ~4-5 GB)
free, total = torch.cuda.mem_get_info()
free_gb  = free / (1024**3)
total_gb = total / (1024**3)

# Máximo 3 modelos; si VRAM libre <7.5 GB, usa 2; si >=11 GB, prueba 3
MAX_MODELS = 3 if free_gb >= 11 else (2 if free_gb >= 7.5 else 1)
print(f"GPU VRAM libre: {free_gb:.1f} GB / {total_gb:.1f} GB → instancias: {MAX_MODELS}")

# ───── UTILIDADES ───────────────────────────────────────────────────────────────
def gather_audio_files(root: Path):
    for dp, _, files in os.walk(root):
        for f in files:
            if Path(f).suffix.lower() in AUDIO_EXTS:
                yield Path(dp) / f

def transcribe_with_model(model: WhisperModel, audio_path: Path, out_dir: Path) -> str:
    out_path = out_dir / (audio_path.stem + ".txt")
    if out_path.exists():
        return f"➡️ {audio_path.name} (ya existe)"
    segments, _ = model.transcribe(
        str(audio_path),
        language=LANG,
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=400),
        beam_size=BEAM_SIZE,
        best_of=1,
        temperature=0.0,
        without_timestamps=True,
        word_timestamps=False,
        condition_on_previous_text=False,
    )
    text = "".join(s.text for s in segments).strip()
    out_path.write_text(text, encoding="utf-8")
    return f"✔️ {audio_path.name} ({len(text)} chars)"

# ───── POOL DE WORKERS (CADA UNO CON SU MODELO) ────────────────────────────────
def worker_loop(worker_id: int, q: Queue, results: list, lock: threading.Lock):
    # Cada worker carga SU propia instancia del modelo (mantiene GPU ocupada)
    model = WhisperModel(
        MODEL_NAME,
        device="cuda",
        compute_type=COMPUTE_TYPE,
        device_index=0,      # cambia si usas otra GPU
        num_workers=1        # hilos internos del modelo (no subir en GPU)
    )
    while True:
        audio_path = q.get()
        if audio_path is None:
            q.task_done()
            break
        try:
            msg = transcribe_with_model(model, audio_path, OUTPUT_DIR)
        except Exception as e:
            msg = f"❌ {audio_path.name} → {e}"
        with lock:
            results.append(msg)
            print(msg)
        q.task_done()

def main():
    files = list(gather_audio_files(INPUT_ROOT))
    total = len(files)
    if total == 0:
        print("⚠️ No se encontraron audios.")
        return

    # Ordenar por tamaño descendente para balancear carga (los largos primero)
    files.sort(key=lambda p: p.stat().st_size if p.exists() else 0, reverse=True)

    print(f"🗂️  Audios: {total}  ·  Workers(modelos): {MAX_MODELS}\n")

    q = Queue(maxsize=MAX_MODELS * 4)  # prefetched items
    results, lock = [], threading.Lock()
    threads = []

    # Lanzar N workers (cada uno con su modelo)
    for i in range(MAX_MODELS):
        t = threading.Thread(target=worker_loop, args=(i, q, results, lock), daemon=True)
        t.start()
        threads.append(t)

    # Ingesta en la cola con barra de progreso simple
    start = time.time()
    for idx, f in enumerate(files, 1):
        q.put(f)
        if idx % 50 == 0:
            elapsed = time.time() - start
            done = len(results)
            if done:
                rate = done / elapsed               # archivos/seg
                eta  = (total - done) / rate
                print(f"⏳ {done}/{total} · {rate*60:.1f} a/min · ETA {eta/60:.1f} min")

    # Señales de fin
    for _ in range(MAX_MODELS):
        q.put(None)

    q.join()  # esperar a que terminen
    
    dur = time.time() - start
    print(f"\n🏁 Listo: {len(results)}/{total} en {dur/60:.1f} min "
          f"({(total/(dur/60)):0.1f} audios/min aprox)")

if __name__ == "__main__":
    main()


GPU VRAM libre: 10.8 GB / 12.0 GB → instancias: 2
🗂️  Audios: 3840  ·  Workers(modelos): 2

✔️ 1766512671.305881.WAV (9632 chars)
✔️ 1765551123.284033.WAV (17156 chars)
✔️ 1766583381.307572.WAV (9072 chars)
✔️ 1765555797.284588.WAV (5530 chars)
✔️ 1765552157.284196.WAV (8452 chars)
✔️ 1765560188.285035.WAV (6641 chars)
✔️ 1765637129.286134.WAV (10461 chars)
✔️ 1766778951.310955.WAV (5068 chars)
✔️ 1766590098.307901.WAV (9699 chars)
✔️ 1767128118.318350.WAV (8239 chars)
✔️ 1765810484.288011.WAV (6458 chars)
✔️ 1767110881.317786.WAV (4875 chars)
✔️ 1766162157.298248.WAV (5514 chars)
✔️ 1766010724.294635.WAV (4512 chars)
✔️ 1765804448.287463.WAV (9127 chars)
✔️ 1766760025.309647.WAV (4556 chars)
✔️ 1767029734.315426.WAV (7377 chars)
✔️ 1766600393.308498.WAV (2621 chars)
✔️ 1766770291.310596.WAV (6857 chars)
✔️ 1767375003.320042.WAV (5267 chars)
✔️ 1765559959.285001.WAV (7242 chars)
✔️ 1766437751.304083.WAV (6027 chars)
✔️ 1766584279.307604.WAV (3588 chars)
✔️ 1766242269.299734.WAV (6989 c

## Evaluaciond e los txt ventas


In [ ]:
import re
import unicodedata
from pathlib import Path
import pandas as pd

# ===================== RUTA DE ENTRADA =====================
# ← CAMBIA AQUÍ
INPUT_TXT_DIR = Path(r"C:\RUTA\A\LOS\TXT")

# =============== Diccionario de Categorías ===============
diccionarios = {
    "saludo_inicial": [
        "hola", "buenos dias", "buenas tardes", "buenas noches", "bienvenido a cl tiene",
        "habla con", "le saluda", "mi nombre es", "en que puedo ayudarle", "como esta",
        "llamo de cltiene", "gracias por comunicarse"
    ],
    "identificacion_cliente": [
        "me confirma su nombre", "cual es su nombre completo", "numero de cedula",
        "con quien tengo el gusto", "verificar informacion", "es usted el titular"
    ],
    "comprension_problema": [
        "cuenteme que sucedio", "cual es el motivo de su llamada", "necesita asistencia",
        "tiene algun inconveniente", "entiendo", "comprendo"
    ],
    "ofrecimiento_solucion": [
        "podemos ofrecerle", "tenemos disponible", "le puedo brindar", "ambulancia",
        "medico a domicilio", "plomeria", "cerrajeria", "veterinario"
    ],
    "manejo_inquietudes": [
        "tiene alguna duda", "alguna pregunta", "no se preocupe", "estamos para ayudarle"
    ],
    "cierre_servicio": [
        "fue un gusto atenderle", "gracias por llamar", "que tenga un buen dia",
        "quedamos atentos"
    ],
    "proximo_paso": [
        "el tecnico se comunicara", "recibira una confirmacion", "por whatsapp", "el proximo paso es"
    ]
}

# ===================== Funciones =====================
def leer_txt(path: Path) -> str:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            return path.read_text(encoding=enc, errors="ignore")
        except:
            continue
    return ""

def normalizar_texto(texto: str) -> str:
    texto = texto.lower()
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8")
    texto = re.sub(r"[^\w\s]", " ", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

def compilar_patrones(dic):
    patrones = {}
    for cat, frases in dic.items():
        frases_norm = [normalizar_texto(f) for f in frases]
        pattern = "|".join(map(re.escape, frases_norm))
        patrones[cat] = re.compile(pattern, flags=re.IGNORECASE)
    return patrones

def contar(texto, patrones):
    return {cat: len(rx.findall(texto)) for cat, rx in patrones.items()}

# ===================== PROCESO =====================
def procesar():
    txt_files = list(Path(INPUT_TXT_DIR).rglob("*.txt"))
    if not txt_files:
        print("⚠️ No se encontraron archivos TXT en la carpeta.")
        return

    patrones = compilar_patrones(diccionarios)
    registros = []

    for archivo in txt_files:
        raw = leer_txt(archivo)
        texto = normalizar_texto(raw)
        conteos = contar(texto, patrones)
        conteos["archivo"] = archivo.name
        registros.append(conteos)

    df = pd.DataFrame(registros)
    print("✅ Procesamiento completado.")
    print(df.head())

    # Guardar
    salida = Path(INPUT_TXT_DIR) / f"evaluacion_cltiene_{len(df)}_llamadas.xlsx"
    df.to_excel(salida, index=False)
    print(f"📂 Archivo generado: {salida}")

    return df

if __name__ == "__main__":
    procesar()


## codigo que evalua llamdas ventas 

In [6]:
# -*- coding: utf-8 -*-
# === Evaluación de llamadas (TXT + OLLAMA) + METADATA EXCEL → 1 Excel con todas las filas ===

import os, re, json, requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd

# ---- Sentimiento clásico (opcional). Si no está, el script sigue funcionando.
try:
    from textblob import TextBlob
    from textblob.classifiers import NaiveBayesClassifier
    _HAS_TB = True
except Exception:
    _HAS_TB = False

# ===================== CONFIGURACIÓN =====================
# Carpeta donde están TODOS los .txt transcritos
INPUT_TXT_DIR = Path(r"C:\descargas\Contactvox_ultimo33333_SERVICIO_1_07_2026\txt")   # <<--- CAMBIA AQUÍ

# Archivo Excel con el detalle de las llamadas (como "Llamadas salientes Noviembre ventas y servicio.xlsx")
EXCEL_METADATA = Path(r"C:\Users\haider_bello\Downloads\Información llamadas de venta y de servicio del 12 de Diciembre al 2 de Enero.xlsx")  # <<--- CAMBIA AQUÍ
EXCEL_SHEET    = "Servicio"  # hoja donde están las llamadas

EXPORT_XLSX   = None  # si None => evaluacion_ollama_<N>_llamadas.xlsx en la misma carpeta TXT
CONTEXTO      = "Servicio"  # "ventas" o "servicio" (se usa también en la columna 'tipo')

# Ollama
OLLAMA_URL    = os.environ.get("OLLAMA_URL", "http://localhost:11434")
MODEL         = os.environ.get("OLLAMA_MODEL", "qwen2.5:7b-instruct")  # ej.: "qwen2.5:7b-instruct", "llama3.1:8b"
TIMEOUT       = 120
RETRIES       = 2
MAX_WORKERS   = int(os.environ.get("MAX_WORKERS", "4"))

# Opciones deterministas (clasificación estable)
OLLAMA_OPTIONS = {
    "temperature": 0,
    "top_p": 0.0,
    "num_thread": 8,
    "num_ctx": 4096,
    "gpu_layers": "auto",
}

# ===================== CATEGORÍAS =====================
CATEGORIAS = [
    "saludo_inicial",
    "identificacion_cliente",
    "comprension_problema",
    "ofrecimiento_solucion",
    "manejo_inquietudes",
    "cierre_servicio",
    "proximo_paso",
]
N_CATS = len(CATEGORIAS)

# ===================== COLUMNAS DE SALIDA =====================
# Columnas que vienen del Excel (mismas que tienes en "Ventas")
META_COLS = [
    "Fecha",
    "Contacto (Identificacion - Nombre",
    "Telefono",
    "Agente",
    "Cuenta",
    "Modulo",
    "Nombre del Modulo",
    "Motivo",
    "Estado de la LLamada",
    "Tiempo  de Llamada",
    "Tiempo  de Conversacion",
    "Estado de Registro",
    "Estado de Gestion",
    "Calificacion de la Llamada",
]

# Columnas de direcciones de grabación que quieres en el Excel final
DIR_COLS = [
    "Direccion grabacion#1",
    "Direccion grabacion#2",
    "Direccion grabacion#3",
    "Direccion grabacion#4",
    "Direccion grabacion#5",
    "Direccion grabacion#6",
    "Direccion grabacion#7",
]

# ===================== PROMPT =====================
def build_prompt(texto: str) -> str:
    contexto_msg = (
        "Contexto: es una llamada de VENTAS. Valora ofrecimiento, manejo de objeciones y cierre orientado a compra.\n"
        if CONTEXTO.lower() == "ventas" else
        "Contexto: es una llamada de SERVICIO/ATENCIÓN. Valora diagnóstico del caso, información clara y siguiente paso operativo.\n"
    )
    return (
        contexto_msg +
        "Eres evaluador de calidad de llamadas. Lee la transcripción y devuelve SOLO un objeto JSON válido "
        "con EXACTAMENTE estas claves, usando 1 si la categoría aparece de forma clara y 0 si no:\n"
        f"{CATEGORIAS}\n\n"
        "Definiciones breves:\n"
        "- saludo_inicial: saludo/presentación del asesor.\n"
        "- identificacion_cliente: pide/confirma datos del cliente (nombre, documento, titularidad, etc.).\n"
        "- comprension_problema: explora el motivo/necesidad del cliente.\n"
        "- ofrecimiento_solucion: propone servicios/soluciones concretas.\n"
        "- manejo_inquietudes: atiende dudas/objeciones (precio, cobertura, condiciones, etc.).\n"
        "- cierre_servicio: cierre/agradecimiento/confirmación final.\n"
        "- proximo_paso: indica acción siguiente (agendamiento, confirmación por WhatsApp/correo, técnico se comunica, etc.).\n\n"
        "IMPORTANTE: Responde SOLO JSON válido, sin texto adicional, usando enteros 0/1.\n\n"
        "Transcripción a evaluar:\n"
        "------------------------\n"
        f"{texto}\n"
        "------------------------\n"
    )

# ===================== UTILIDADES =====================
def check_ollama():
    r = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
    r.raise_for_status()
    print("✅ Ollama responde en", OLLAMA_URL)

def leer_txt(path: Path) -> str:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            return path.read_text(encoding=enc, errors="ignore")
        except Exception:
            continue
    return ""

def extract_json(s: str):
    """Intenta recuperar el primer objeto JSON de la respuesta del modelo."""
    s = (s or "").strip()
    if s.startswith("{") and s.endswith("}"):
        try:
            return json.loads(s)
        except Exception:
            pass
    start, end = s.find("{"), s.rfind("}")
    if start != -1 and end != -1 and end > start:
        try:
            return json.loads(s[start:end+1])
        except Exception:
            pass
    m = re.search(r"\{.*\}", s, re.DOTALL)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            pass
    return None

def extract_id_audio_from_filename(path: Path):
    """
    Extrae el ID de audio del nombre del archivo .txt.
    Ejemplo:
      'David_Alejandro_Paloma_Murillo_20251108_111150_1762618310.232720.txt'
      --> '1762618310.232720'
    """
    m = re.search(r"(\d+\.\d+)", path.stem)
    if m:
        return m.group(1)
    return None

# ---- métricas auxiliares (polarity/subjectivity/clasificación/confianza/palabras) ----
if _HAS_TB:
    _NB = NaiveBayesClassifier([
        ("Perfecto, entonces quedamos así", "positivo"),
        ("Sí, me interesa el producto", "positivo"),
        ("Gracias por la ayuda", "positivo"),
        ("No me interesa en este momento", "negativo"),
        ("Estoy inconforme con la atención", "negativo"),
    ])

def sentiment_metrics(texto: str):
    texto = texto or ""
    if not _HAS_TB:
        pol = 0.0
        sub = 0.0
        conf = 0.5
    else:
        try:
            tb = TextBlob(texto)
            pol, sub = float(tb.sentiment.polarity), float(tb.sentiment.subjectivity)
        except Exception:
            pol, sub = 0.0, 0.0
        try:
            d = _NB.prob_classify(texto)
            conf = float(d.prob(d.max()))
        except Exception:
            conf = 0.5 + 0.5 * min(1.0, abs(pol))

    clasif = "positivo" if pol >= 0.1 else ("negativo" if pol <= -0.1 else "neutro")
    palabras = len(re.findall(r"\b\w+\b", texto, flags=re.UNICODE))
    return pol, sub, clasif, conf, palabras

# ===================== Llamada a OLLAMA =====================
def classify_text(texto: str, retries: int = RETRIES) -> dict:
    """Devuelve {cat: 0/1} para todas las categorías."""
    texto = "" if texto is None else str(texto).strip()
    if not texto:
        return {k: 0 for k in CATEGORIAS}

    prompt = build_prompt(texto)
    payload = {"model": MODEL, "prompt": prompt, "stream": False, "options": OLLAMA_OPTIONS}
    last_err = None
    for _ in range(retries + 1):
        try:
            resp = requests.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=TIMEOUT)
            resp.raise_for_status()
            data = resp.json().get("response", "")
            j = extract_json(data) or {}
            limpio = {k: int(j.get(k, 0)) for k in CATEGORIAS}
            for k in limpio:
                limpio[k] = 1 if limpio[k] else 0
            return limpio
        except Exception as e:
            last_err = e
    # Fallback: todo en 0 si falla
    return {k: 0 for k in CATEGORIAS}

# ===================== CARGA DE METADATOS (EXCEL) =====================
def cargar_metadata():
    """
    Lee el Excel de llamadas y devuelve:
      df_meta: DataFrame original + columna 'id_audio'
      meta_by_id: DataFrame indexado por 'id_audio' para merges rápidos
    """
    if not EXCEL_METADATA.exists():
        raise FileNotFoundError(f"No se encontró el Excel de metadata: {EXCEL_METADATA}")

    df_meta = pd.read_excel(EXCEL_METADATA, sheet_name=EXCEL_SHEET)
    # id_audio se toma de la columna 'nombre_audio' (ej: 1762618310.232720.WAV --> 1762618310.232720)
    if "nombre_audio" not in df_meta.columns:
        raise KeyError("El Excel debe tener una columna llamada 'nombre_audio'.")

    df_meta["id_audio"] = df_meta["nombre_audio"].astype(str).str.extract(r"(\d+\.\d+)", expand=False)
    meta_by_id = (
        df_meta
        .dropna(subset=["id_audio"])
        .drop_duplicates("id_audio")
        .set_index("id_audio")
    )
    print(f"📊 Metadata cargada: {len(df_meta)} filas, {len(meta_by_id)} IDs únicos de audio.")
    return df_meta, meta_by_id

def extraer_direcciones(meta_row):
    """
    A partir de una fila del Excel (meta_row), arma un dict con:
    Direccion grabacion#1 ... Direccion grabacion#7
    Soporta dos casos:
      - Columnas numeradas (Direccion grabacion#1, #2, ...)
      - Solo una columna 'Direccion grabacion' (se pone en #1)
    """
    dirs = [None] * 7
    if meta_row is not None:
        # Caso 1: ya existen las columnas numeradas en el Excel
        any_found = False
        for i in range(7):
            col_name = f"Direccion grabacion#{i+1}"
            if col_name in meta_row.index:
                dirs[i] = meta_row.get(col_name)
                any_found = True

        # Caso 2: si no hay numeradas, usar 'Direccion grabacion' como #1
        if not any_found and "Direccion grabacion" in meta_row.index:
            dirs[0] = meta_row.get("Direccion grabacion")

    return {f"Direccion grabacion#{i+1}": dirs[i] for i in range(7)}

# ===================== PIPELINE PRINCIPAL =====================
def evaluar_carpeta_txt():
    check_ollama()

    # 1) Cargar metadata del Excel
    df_meta, meta_by_id = cargar_metadata()

    # 2) Descubrir .txt
    txt_files = sorted(Path(INPUT_TXT_DIR).rglob("*.txt"))
    if not txt_files:
        print(f"⚠️ No se encontraron .txt en: {INPUT_TXT_DIR}")
        cols_final = (
            META_COLS
            + DIR_COLS
            + ["Comentario", "archivo"]
            + CATEGORIAS
            + ["efectiva", "polarity", "subjectivity", "clasificacion", "confianza", "palabras", "tipo"]
        )
        return pd.DataFrame(columns=cols_final)

    print(f"📂 Archivos a procesar: {len(txt_files)}  ·  modelo={MODEL}  ·  workers={MAX_WORKERS}")

    registros = []

    def _proc(path: Path):
        raw = leer_txt(path)
        cats = classify_text(raw)
        # Puntaje simple: proporción de categorías marcadas
        puntaje = sum(cats.values()) / float(N_CATS or 1)
        efectiva = 1 if puntaje >= 0.8 else 0  # umbral ajustable
        pol, sub, clasif, conf, palabras = sentiment_metrics(raw)

        # Vincular con Excel usando el ID de audio en el nombre del TXT
        id_audio = extract_id_audio_from_filename(path)
        meta_row = meta_by_id.loc[id_audio] if id_audio and id_audio in meta_by_id.index else None

        base = {col: None for col in META_COLS}  # inicio con metadatos vacíos

        if meta_row is not None:
            # Copiar solo las columnas de interés (si existen en el Excel)
            for col in META_COLS:
                if col in meta_row.index:
                    base[col] = meta_row.get(col)

        # Direcciones de grabación
        base.update(extraer_direcciones(meta_row))

        # Comentario (si existiera en el Excel, lo usamos; si no, queda vacío)
        comentario = None
        if meta_row is not None and "Comentario" in meta_row.index:
            comentario = meta_row.get("Comentario")

        base.update({
            "Comentario": comentario,
            "archivo": path.name,
            **cats,
            "efectiva": efectiva,
            "polarity": round(pol, 3),
            "subjectivity": round(sub, 3),
            "clasificacion": clasif,
            "confianza": round(conf, 2),
            "palabras": palabras,
            "tipo": CONTEXTO.lower(),
        })

        return base

    # 3) Paralelizar procesamiento
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs = {ex.submit(_proc, p): p for p in txt_files}
        done = 0
        step = max(1, min(25, len(futs) // 10 or 1))
        for fut in as_completed(futs):
            registros.append(fut.result())
            done += 1
            if done % step == 0 or done == len(futs):
                print(f"⏳ Progreso: {done}/{len(futs)}", flush=True)

    # 4) Armar DataFrame final en el orden que quieres
    cols_final = (
        META_COLS
        + DIR_COLS
        + ["Comentario", "archivo"]
        + CATEGORIAS
        + ["efectiva", "polarity", "subjectivity", "clasificacion", "confianza", "palabras", "tipo"]
    )

    df = pd.DataFrame(registros).reindex(columns=cols_final)

    # 5) Exportar 1 solo Excel con todas las filas
    fname = EXPORT_XLSX or f"evaluacion_ollama_{len(df)}_llamadas_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    out_path = Path(INPUT_TXT_DIR) / fname
    df.to_excel(out_path, index=False)

    print(f"✅ Excel generado: {out_path}")
    print(df.head(5))
    return df

# ===================== MAIN =====================
if __name__ == "__main__":
    evaluar_carpeta_txt()


✅ Ollama responde en http://localhost:11434
📊 Metadata cargada: 1059 filas, 1059 IDs únicos de audio.
📂 Archivos a procesar: 1019  ·  modelo=qwen2.5:7b-instruct  ·  workers=4
⏳ Progreso: 25/1019
⏳ Progreso: 50/1019
⏳ Progreso: 75/1019
⏳ Progreso: 100/1019
⏳ Progreso: 125/1019
⏳ Progreso: 150/1019
⏳ Progreso: 175/1019
⏳ Progreso: 200/1019
⏳ Progreso: 225/1019
⏳ Progreso: 250/1019
⏳ Progreso: 275/1019
⏳ Progreso: 300/1019
⏳ Progreso: 325/1019
⏳ Progreso: 350/1019
⏳ Progreso: 375/1019
⏳ Progreso: 400/1019
⏳ Progreso: 425/1019
⏳ Progreso: 450/1019
⏳ Progreso: 475/1019
⏳ Progreso: 500/1019
⏳ Progreso: 525/1019
⏳ Progreso: 550/1019
⏳ Progreso: 575/1019
⏳ Progreso: 600/1019
⏳ Progreso: 625/1019
⏳ Progreso: 650/1019
⏳ Progreso: 675/1019
⏳ Progreso: 700/1019
⏳ Progreso: 725/1019
⏳ Progreso: 750/1019
⏳ Progreso: 775/1019
⏳ Progreso: 800/1019
⏳ Progreso: 825/1019
⏳ Progreso: 850/1019
⏳ Progreso: 875/1019
⏳ Progreso: 900/1019
⏳ Progreso: 925/1019
⏳ Progreso: 950/1019
⏳ Progreso: 975/1019
⏳ Progres